# Phishing Response Automation Demo

**Scenario (from the webinar):** A SOC agent triages a reported phishing email and triggers containment. The demo exposes a single MCP tool backed by an Orkes Conductor workflow that creates a case, enriches context, and returns a case ID + status.

This notebook shows the end-to-end flow with **MCP Workbench (local)** and **Orkes Conductor (cloud)**.


## Demo flow
1. Register a phishing-response workflow in Orkes Conductor.
2. Configure MCP Gateway (service + route + schema).
3. Use MCP Workbench to list tools and run the phishing-response tool.
4. Validate output (case ID + status) and check execution in Conductor.


In [ ]:
import json
import os
import time
import urllib.request
import urllib.error

def require_env(name, default=None):
    value = os.environ.get(name, default)
    if value is None or value == "":
        raise RuntimeError(f"Set {name} in your environment before running this cell.")
    return value

ORKES_SERVER_URL = os.environ.get("ORKES_SERVER_URL", "https://developer.orkescloud.com/api")
ORKES_KEY_ID = require_env("ORKES_KEY_ID")
ORKES_KEY_SECRET = require_env("ORKES_KEY_SECRET")
ORKES_APPLICATION_NAME = os.environ.get("ORKES_APPLICATION_NAME", "mcp-workbench-gateway-demo")
MCP_TOOL_ENDPOINT = os.environ.get("MCP_TOOL_ENDPOINT")  # set after MCP Gateway is configured

def http_json(method, url, body=None, headers=None, timeout=30):
    headers = headers or {}
    data = None
    if body is not None:
        data = json.dumps(body).encode("utf-8")
        headers.setdefault("Content-Type", "application/json")
    req = urllib.request.Request(url, data=data, method=method)
    for k, v in headers.items():
        req.add_header(k, v)
    try:
        with urllib.request.urlopen(req, timeout=timeout) as resp:
            raw = resp.read().decode("utf-8")
            if not raw:
                return {}
            try:
                return json.loads(raw)
            except json.JSONDecodeError:
                return raw
    except urllib.error.HTTPError as e:
        raw = e.read().decode("utf-8")
        raise RuntimeError(f"HTTP {e.code}: {raw}") from e

def get_jwt():
    body = {"keyId": ORKES_KEY_ID, "keySecret": ORKES_KEY_SECRET}
    resp = http_json("POST", f"{ORKES_SERVER_URL}/token", body, headers={"accept": "application/json"})
    return resp["token"]


## 1) Register the phishing-response workflow in Orkes Conductor
This workflow creates a mock case ID, assigns severity, and returns a compact response object (caseId, status, severity, ioc, summary).


In [ ]:
workflow_name = "phishing_response_wf"

workflow_def = {
    "name": workflow_name,
    "description": "Phishing response workflow for MCP demo",
    "version": 1,
    "schemaVersion": 2,
    "inputParameters": [
        "report_id",
        "reporter_email",
        "subject",
        "sender",
        "suspicious_url",
        "priority",
        "summary",
        "source",
    ],
    "tasks": [
        {
            "name": "triage_case",
            "taskReferenceName": "triage_case",
            "type": "INLINE",
            "inputParameters": {
                "priority": "${workflow.input.priority}",
                "sender": "${workflow.input.sender}",
                "suspicious_url": "${workflow.input.suspicious_url}",
                "report_id": "${workflow.input.report_id}",
                "summary": "${workflow.input.summary}",
                "evaluatorType": "graaljs",
                "expression": (
                    "(function(){"
                    "var sev=(function(p){if(!p) return 'medium'; p=p.toLowerCase(); if(p==='high'||p==='critical') return 'high'; if(p==='low') return 'low'; return 'medium';})( $.priority );"
                    "var caseId='PHISH-'+Math.floor(Math.random()*900000+100000);"
                    "return {caseId:caseId,status:'CONTAINMENT_STARTED',severity:sev,ioc:{sender:$.sender,url:$.suspicious_url},summary:$.summary||('Phishing report '+($.report_id||''))};"
                    "})()"
                ),
            },
        },
        {
            "name": "prepare_output",
            "taskReferenceName": "prepare_output",
            "type": "JSON_JQ_TRANSFORM",
            "inputParameters": {
                "source": {
                    "caseId": "${triage_case.output.result.caseId}",
                    "status": "${triage_case.output.result.status}",
                    "severity": "${triage_case.output.result.severity}",
                    "ioc": "${triage_case.output.result.ioc}",
                    "summary": "${triage_case.output.result.summary}",
                },
                "queryExpression": "{caseId: .source.caseId, status: .source.status, severity: .source.severity, ioc: .source.ioc, summary: .source.summary}",
            },
        },
    ],
    "outputParameters": {
        "caseId": "${prepare_output.output.result.caseId}",
        "status": "${prepare_output.output.result.status}",
        "severity": "${prepare_output.output.result.severity}",
        "ioc": "${prepare_output.output.result.ioc}",
        "summary": "${prepare_output.output.result.summary}",
    },
    "ownerEmail": "demo@orkes.io",
    "timeoutPolicy": "ALERT_ONLY",
    "timeoutSeconds": 0,
    "restartable": True,
    "workflowStatusListenerEnabled": False,
}

token = get_jwt()
headers = {"X-Authorization": token, "accept": "*/*"}
resp = http_json(
    "POST",
    f"{ORKES_SERVER_URL}/metadata/workflow?overwrite=true&newVersion=false",
    workflow_def,
    headers=headers,
)
print("Workflow registered:", workflow_name)


In [ ]:
sample_input = {
    "report_id": "INC-2026-0017",
    "reporter_email": "alex.soc@example.com",
    "subject": "Urgent: Password reset required",
    "sender": "it-support@evil.example",
    "suspicious_url": "http://evil.example/reset",
    "priority": "high",
    "summary": "Employee reported a suspicious password reset email.",
    "source": "soc-agent",
}

start_resp = http_json(
    "POST",
    f"{ORKES_SERVER_URL}/workflow/{workflow_name}",
    sample_input,
    headers={"X-Authorization": token},
)
workflow_id = start_resp.get("workflowId") if isinstance(start_resp, dict) else start_resp
print("Started workflow:", workflow_id)

# Poll for completion and print output
status = None
for _ in range(20):
    status = http_json(
        "GET",
        f"{ORKES_SERVER_URL}/workflow/{workflow_id}",
        headers={"X-Authorization": token},
    )
    if status.get("status") in ("COMPLETED", "FAILED", "TERMINATED", "TIMED_OUT"):
        break
    time.sleep(1)

print("Status:", status.get("status"))
print("Output:", json.dumps(status.get("output", {}), indent=2))


## 2) Configure MCP Gateway (UI steps)
Use Orkes Conductor UI to expose the workflow as an MCP tool. Follow the same flow as the MCP Gateway tutorial (create schema, create app with EXECUTE permission, configure auth, create service + route, and copy the MCP Tool Remote Endpoint).

Below is a compact input schema for this phishing-response tool. Use it in **Definitions > Schema** and attach it to your MCP route.


In [ ]:
phishing_schema = {
  "name": "PhishingResponseSchema",
  "version": 1,
  "type": "JSON",
  "data": {
    "$schema": "http://json-schema.org/draft-07/schema",
    "type": "object",
    "properties": {
      "report_id": {"type": "string", "description": "Incident or report identifier."},
      "reporter_email": {"type": "string", "format": "email", "description": "Reporter email."},
      "subject": {"type": "string", "description": "Email subject line."},
      "sender": {"type": "string", "description": "Email sender."},
      "suspicious_url": {"type": "string", "format": "uri", "description": "Suspicious URL to investigate."},
      "priority": {"type": "string", "enum": ["low", "medium", "high", "critical"], "description": "Urgency level."},
      "summary": {"type": "string", "description": "Short summary of the report."},
      "source": {"type": "string", "description": "Where the request originated (agent, chatbot, UI)."}
    },
    "required": ["report_id", "reporter_email", "subject", "sender", "suspicious_url", "priority", "source"]
  }
}

print(json.dumps(phishing_schema, indent=2))


## 3) Call the MCP tool (JSON-RPC over Streamable HTTP)
Set `MCP_TOOL_ENDPOINT` in your environment after you create the MCP route (copy the **MCP Tool Remote Endpoint** from the service configuration).

This mirrors what MCP Workbench does: list tools and invoke the phishing-response tool.


In [ ]:
if not MCP_TOOL_ENDPOINT:
    raise RuntimeError("Set MCP_TOOL_ENDPOINT before running this cell.")

MCP_AUTH_HEADER = os.environ.get("MCP_AUTH_HEADER")
MCP_AUTH_VALUE = os.environ.get("MCP_AUTH_VALUE")

def mcp_call(endpoint, method, params=None, request_id=1):
    payload = {"jsonrpc": "2.0", "id": request_id, "method": method}
    if params is not None:
        payload["params"] = params
    headers = {"Accept": "application/json"}
    if MCP_AUTH_HEADER and MCP_AUTH_VALUE:
        headers[MCP_AUTH_HEADER] = MCP_AUTH_VALUE
    return http_json("POST", endpoint, payload, headers=headers)

tools = mcp_call(MCP_TOOL_ENDPOINT, "tools/list")
tool_list = tools.get("result", {}).get("tools", [])
print("Tools:", [t.get("name") for t in tool_list])

# Try to pick the phishing tool automatically; otherwise set the name manually
tool_name = None
for t in tool_list:
    name = t.get("name", "")
    if "phish" in name.lower() or "triage" in name.lower():
        tool_name = name
        break

if not tool_name:
    raise RuntimeError("Could not infer tool name. Set tool_name manually.")

call_resp = mcp_call(
    MCP_TOOL_ENDPOINT,
    "tools/call",
    {"name": tool_name, "arguments": sample_input},
    request_id=2,
)
print(json.dumps(call_resp, indent=2))


## 4) Validate in MCP Workbench (locally hosted)
Use MCP Workbench to reproduce, validate, and inspect the request/response:
1. Open your locally hosted MCP Workbench.
2. Add a connection with **Type = Streamable HTTP (Stateless)** and URL = your MCP Tool Remote Endpoint.
3. Connect, then **List tools** and select the phishing-response tool.
4. Run the tool with the sample input and verify caseId + status, then check the workflow execution in Orkes Conductor.


In [ ]:
# Optional negative test (schema validation)
# Remove a required field and confirm the MCP Gateway rejects the request with a clear error.
bad_input = dict(sample_input)
bad_input.pop("reporter_email", None)

try:
    bad_resp = mcp_call(
        MCP_TOOL_ENDPOINT,
        "tools/call",
        {"name": tool_name, "arguments": bad_input},
        request_id=3,
    )
    print(json.dumps(bad_resp, indent=2))
except RuntimeError as e:
    print("Expected validation error:", e)
